# 30m DEM analysis of the Mataquito

In [1]:
from TopoAnalysis import dem as d

In [5]:
elevation = d.Elevation.load('dem_utm30m.tif')
print(elevation._georef_info.xllcenter, elevation._georef_info.yllcenter)

192544.10536118236 6051720.538252637


In [3]:
import os

print(f"Current working directory: {os.getcwd()}")
print(f"Is file accessible? {os.path.isfile('dem_utm30m.tif')}")

Current working directory: /Users/Glong1/Desktop/Andes/Andes_watersheds/MataquitoRiver
Is file accessible? False


In [ ]:
# Set the working directory to where your file is located
os.chdir('/Users/Glong1/Desktop/Andes/Andes_watersheds/MataquitoRiver/30DEMwatersheddata')

# Verify the working directory
print(f"New working directory: {os.getcwd()}")

In [6]:
filled = d.FilledElevation(elevation = elevation)

In [7]:
flow_direction = d.FlowDirectionD8(flooded_dem = filled)
area = d.Area(flow_direction = flow_direction)
log_area = d.LogArea(area = area)

In [8]:
filled.save('dem_utm_filled30m')
flow_direction.save('fd_utm30m')
area.save('area_utm30m')
log_area.save('logarea_utm30m')

In [ ]:
"""CT-8.2	5-11-24	35°05'32''S	71.647745°W
CT-9	5-7-24	35°02'47''S	72°06'04''W
CT-10	5-7-24	35°02'58''S	71°32'60''W
CT-11	5-8-24	35°03'51''S	71°36'37''W
"""

In [ ]:
## Convert Lat long to easting northing
from pyproj import Transformer

def lat_long_to_utm_epsg32719(lat, lon):
    """
    Converts latitude and longitude to UTM easting and northing using EPSG: . . ..

    Parameters:
        lat (float): Latitude in decimal degrees.
        lon (float): Longitude in decimal degrees.

    Returns:
        tuple: (easting, northing) in meters.
    """
    # Define the transformer for EPSG:4326 (WGS 84) to EPSG:32719 (WGS 84 / UTM Zone 19S)
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:32719", always_xy=True)
    # Transform coordinates
    easting, northing = transformer.transform(lon, lat)
    return easting, northing

# Coords
coordinates = [
    (-71.647745, -35.092222),
    (-72.101111, -35.046389),
    (-71.550000, -35.049444),
    (-71.610278, -35.064167),
]

# Perform conversion and print results
for lon, lat in coordinates:
    easting, northing = lat_long_to_utm_epsg32719(lat, lon)
    print(f"{easting:.2f}, {northing:.2f}")

In [ ]:
# these are 'new' points based on the 30m DEM now available (the correction is to the flow line based on
# where the samples were actually taken from 

In [9]:
samples = [['CT-1', 314211, 6096048],
            ['CT-2', 220392, 6118342],
            ['CT-3', 231397, 6111850],
            ['CT-4', 273159, 6120653],
            ['CT-5', 284525, 6128440],
            ['CT-6', 286170, 6126151],
            ['CT-7', 333231, 6125064],
            ['CT-8.2', 258632, 6113638],
            ['CT-9', 217129, 6117568],
            ['CT-10', 267428, 6119008],
            ['CT-11', 262356, 6117038]
           ]

In [10]:
for sample, x, y in samples:
    try:
        basin_mask = d.Mask(flow_direction = flow_direction, outlets = ((x, y),))
        basin_mask.save(sample.replace('-','_') + '_mask')

        print(f"Finished processing sample: {sample}")

    except Exception as e: 
        print(f"failed to process sample {sample}: {e}")

Finished processing sample: CT-1
Finished processing sample: CT-2
Finished processing sample: CT-3
Finished processing sample: CT-4
Finished processing sample: CT-5
Finished processing sample: CT-6
Finished processing sample: CT-7
Finished processing sample: CT-8.2
Finished processing sample: CT-9
Finished processing sample: CT-10
Finished processing sample: CT-11


In [16]:
os.chdir('/Users/Glong1/Desktop/Andes/Andes_watersheds/MataquitoRiver/30DEMwatersheddata/Masks(30m)')

In [18]:
import numpy as np

for sample, _, _ in samples:
    basin_mask = d.Mask.load(sample.replace('-','_') + '_mask')
    total_area = np.sum(basin_mask._griddata)*np.power(basin_mask._georef_info.dx,2)
    print(sample, float(total_area) / 1.0E6, "km2")

CT-1 1384.897766699581 km2
CT-2 5760.468239228807 km2
CT-3 189.37484236164232 km2
CT-4 4706.348116096995 km2
CT-5 1495.573632004314 km2
CT-6 2573.0304360627297 km2
CT-7 1207.494925540769 km2
CT-8.2 4950.141787733108 km2
CT-9 6190.468088718975 km2
CT-10 4864.642928674071 km2
CT-11 4913.059868122383 km2


In [19]:
import os
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

# List of masked raster files (replace with your actual file paths)
masked_rasters = ["CT_1_mask", "CT_2_mask", "CT_3_mask", "CT_4_mask", "CT_5_mask", "CT_6_mask", "CT_7_mask",
                  "CT_8.2_mask", "CT_9_mask", "CT_10_mask", "CT_11_mask"]  # Add your files here

# Output folder for polygon shapefiles
output_folder = "output_polygons"
os.makedirs(output_folder, exist_ok=True)

# Specify the CRS (optional). Example: "EPSG:32719" for UTM Zone 19S
user_defined_crs = "EPSG:32719"  # Change or set to None to use the raster's CRS

# Loop through each masked raster and convert to polygon
for raster_file in masked_rasters:
    try:
        # Open the raster file
        with rasterio.open(raster_file) as src:
            # Read the raster data
            image = src.read(1)  # Read the first band
            mask = image > 0  # Only include non-zero pixels
            
            # Convert raster to polygons
            results = (
                {"properties": {"value": value}, "geometry": shape(geom)}
                for geom, value in shapes(image, mask=mask, transform=src.transform)
            )
            
            # Convert to a GeoDataFrame
            gdf = gpd.GeoDataFrame.from_features(results, crs=src.crs)
            
            # Override CRS if user_defined_crs is provided
            if user_defined_crs:
                gdf = gdf.set_crs(user_defined_crs, allow_override=True)
                print(f"CRS set to: {user_defined_crs}")
            
            # Save as a shapefile
            output_file = os.path.join(output_folder, os.path.basename(raster_file).replace(".tif", "_polygon.shp"))
            gdf.to_file(output_file)
            print(f"Successfully converted {raster_file} to {output_file}")
    
    except Exception as e:
        print(f"Failed to process {raster_file}: {e}")


CRS set to: EPSG:32719
Successfully converted CT_1_mask to output_polygons/CT_1_mask
CRS set to: EPSG:32719
Successfully converted CT_2_mask to output_polygons/CT_2_mask
CRS set to: EPSG:32719
Successfully converted CT_3_mask to output_polygons/CT_3_mask
CRS set to: EPSG:32719
Successfully converted CT_4_mask to output_polygons/CT_4_mask
CRS set to: EPSG:32719
Successfully converted CT_5_mask to output_polygons/CT_5_mask
CRS set to: EPSG:32719
Successfully converted CT_6_mask to output_polygons/CT_6_mask
CRS set to: EPSG:32719
Successfully converted CT_7_mask to output_polygons/CT_7_mask
CRS set to: EPSG:32719
Successfully converted CT_8.2_mask to output_polygons/CT_8.2_mask
CRS set to: EPSG:32719
Successfully converted CT_9_mask to output_polygons/CT_9_mask
CRS set to: EPSG:32719
Successfully converted CT_10_mask to output_polygons/CT_10_mask
CRS set to: EPSG:32719
Successfully converted CT_11_mask to output_polygons/CT_11_mask
